<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/CompLingNoTrainerAllDataWeightedAndBalancedLossAndEvaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd

filepaths = ["/content/ArMIS_train.json", "/content/ConvAbuse_train.json", "/content/HS-Brexit_train.json", "/content/MD-Agreement_train.json", "/content/rom_train.json"]
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs = [df.transpose() for df in dfs]
dfs = [df.astype({"hard_label": int}, errors='raise') for df in dfs]#other way aroud (do concat first) would make more sense

df_all = pd.concat(dfs)
#df_all["soft_label"] =  df_all["soft_label"].apply(lambda x: [x["0"], x["1"]])
#should probably do that, but then evaluate script has to be adapted

# Dataset

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", do_lower_case=True)

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/38685 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [ ]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [ ]:
from torch.utils.data import DataLoader

batch_size = 4
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=2)

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [ ]:
from transformers import get_scheduler
num_epochs = 1
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [ ]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [ ]:
#TODO do this for each dataset independently

#weight = df_all.mean(axis=0)["hard_label"]
weight = df_all["hard_label"].mean()
weight = torch.tensor([weight, 1 - weight], device = device, dtype=torch.float32)#I think this is the right way around

In [ ]:

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]
      #soft_labels = torch.cat([soft_labels["0"].unsqueeze(-1), soft_labels["1"].unsqueeze(-1)], dim=-1).to(device)# that can probably be done more intuitively
      outputs = model(input_ids=inputs, attention_mask = attention_mask)#NllLoss
      optimizer.zero_grad()
      loss = 0
      for j in range(outputs.logits.shape[0]):
        criterion = criterion = torch.nn.CrossEntropyLoss(weight = (torch.tensor([soft_labels["0"][j], soft_labels["1"][j]], device=device) + weight) / 2)#can probably be done more efficient
        loss += criterion(outputs.logits, hard_labels)
      loss = loss/batch_size
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

<ipython-input-59-12eeacebd040>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):


  0%|          | 0/8704 [00:00<?, ?it/s]

[1,    10] loss: 1.084
[1,    20] loss: 0.532
[1,    30] loss: 0.503
[1,    40] loss: 0.663
[1,    50] loss: 0.440
[1,    60] loss: 0.343
[1,    70] loss: 0.469
[1,    80] loss: 0.685
[1,    90] loss: 0.385
[1,   100] loss: 0.505
[1,   110] loss: 0.579
[1,   120] loss: 0.428
[1,   130] loss: 0.774
[1,   140] loss: 0.543
[1,   150] loss: 0.574
[1,   160] loss: 0.581
[1,   170] loss: 0.498
[1,   180] loss: 0.543
[1,   190] loss: 0.605
[1,   200] loss: 0.513
[1,   210] loss: 0.599
[1,   220] loss: 0.319
[1,   230] loss: 0.462
[1,   240] loss: 0.654
[1,   250] loss: 0.530
[1,   260] loss: 0.331
[1,   270] loss: 0.611
[1,   280] loss: 0.375
[1,   290] loss: 0.473
[1,   300] loss: 0.486
[1,   310] loss: 0.257
[1,   320] loss: 0.612
[1,   330] loss: 0.503
[1,   340] loss: 0.452
[1,   350] loss: 0.525
[1,   360] loss: 0.488
[1,   370] loss: 0.445
[1,   380] loss: 0.284
[1,   390] loss: 0.316
[1,   400] loss: 0.552
[1,   410] loss: 0.473
[1,   420] loss: 0.594
[1,   430] loss: 0.582
[1,   440] 

In [ ]:
outputs.logits[0][0].item()

0.8077739477157593

In [ ]:
evaluate_ce_score(model, eval_dataloader)

<ipython-input-57-e001fe9acb63>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/968 [00:00<?, ?it/s]

tensor(0.4401, device='cuda:0')

In [ ]:
evaluate_f1_score_huggingface(model, eval_dataloader)

<ipython-input-57-e001fe9acb63>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/968 [00:00<?, ?it/s]

0.8504648760330579

# Evaluate
create the tsv files

In [ ]:
#load dev data
#create predictions on that data
#save the data in tsv
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import csv
import os


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

batch_size = 1 #can also be implemented with larger batch_size oc

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base", do_lower_case=True)

filepaths_read = ["/content/ArMIS_dev.json", "/content/ConvAbuse_dev.json", "/content/HS-Brexit_dev.json", "/content/MD-Agreement_dev.json",  "/content/rom_dev.json"]
filepaths_write = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv", "/content/romanian_results.tsv"]

epsilon = 1e-12


In [ ]:

#print(filepath_read, "a")
for filepath_read, filepath_write in zip(filepaths_read, filepaths_write):
  if os.path.exists(filepath_write):
    os.remove(filepath_write)

  df = pd.read_json(filepath_read)
  df = df.transpose()

  dataset = Dataset.from_pandas(df[["text"]])
  tokenized_dataset = dataset.map(tokenize_function, batched=False)
  tokenized_dataset.set_format("torch")

  dataloader = DataLoader(tokenized_dataset, shuffle=False, batch_size=batch_size)
  with open(filepath_write, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for batch in tqdm_notebook(dataloader, 0):
      inputs, attention_mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
      with torch.no_grad():
        outputs = model(input_ids=inputs, attention_mask = attention_mask)
      logits = outputs.logits

      prediction = torch.argmax(logits, dim=-1)
      probability = torch.softmax(logits, axis=-1)
      probability = torch.clip(probability, epsilon, 1. - epsilon)
      writer.writerow([prediction[0].item(), probability[0][0].item(), probability[0][1].item()])


  0%|          | 0/141 [00:00<?, ?ex/s]

<ipython-input-64-2c596b60d782>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader, 0):


  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?ex/s]

  0%|          | 0/812 [00:00<?, ?it/s]

...

  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/1104 [00:00<?, ?ex/s]

  0%|          | 0/1104 [00:00<?, ?it/s]

  0%|          | 0/9418 [00:00<?, ?ex/s]

  0%|          | 0/9418 [00:00<?, ?it/s]

In [ ]:
from zipfile import ZipFile

filepath = "xlm-roberta-base.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")
  zipObj.write("romanian_results.tsv")
  

In [ ]:
from google.colab import files
files.download("xlm-roberta-base.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>